# Seleção de variáveis

Neste notebook iremos estudar o dataset __houses__ com o objetivo de avaliar a interação de primeira e segunda ordem que as variáveis causam umas nas outras. Além disso, será aplicado uma seleção de variáveis por meio do método stepwise.

In [1]:
#Leitura e apresentacao do dataset
casas <- read.table("houses.dat", header=T)
head(casas)

case,taxes,beds,baths,new,price,size
1,3104,4,2,0,279.9,2048
2,1173,2,1,0,146.5,912
3,3076,4,2,0,237.7,1654
4,1608,3,2,0,200.0,2068
5,1454,3,3,0,159.9,1477
6,2997,3,2,1,499.9,3153


Ajuste de dois modelos com todos os parâmetros (exceto taxes) avaliando a interação de primeira e segunda ordem.

In [2]:
#Ajustando os modelos
modN1 = with(casas,glm(price ~ beds + baths + new + size, family = gaussian(link="identity")))
modN2 = with(casas,glm(price ~ (beds + baths + new + size)^2, family = gaussian(link="identity")))

#Mostrando dados do modelo
summary(modN1)
summary(modN2)


Call:
glm(formula = price ~ beds + baths + new + size, family = gaussian(link = "identity"))

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-215.747   -30.833    -5.574    18.800   164.471  

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -28.84922   27.26116  -1.058  0.29262    
beds         -8.20238   10.44984  -0.785  0.43445    
baths         5.27378   13.08017   0.403  0.68772    
new          54.56238   19.21489   2.840  0.00553 ** 
size          0.11812    0.01232   9.585 1.27e-15 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 2943.411)

    Null deviance: 1015150  on 99  degrees of freedom
Residual deviance:  279624  on 95  degrees of freedom
AIC: 1089.4

Number of Fisher Scoring iterations: 2



Call:
glm(formula = price ~ (beds + baths + new + size)^2, family = gaussian(link = "identity"))

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-138.303   -28.479    -3.314    18.240   179.437  

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.391e+02  6.662e+01   2.087 0.039727 *  
beds        -5.954e+01  3.330e+01  -1.788 0.077180 .  
baths        1.426e+01  4.703e+01   0.303 0.762432    
new          1.033e+02  1.065e+02   0.970 0.334441    
size        -1.948e-03  5.613e-02  -0.035 0.972395    
beds:baths   8.135e-01  1.750e+01   0.046 0.963028    
beds:new    -1.045e+01  4.017e+01  -0.260 0.795403    
beds:size    3.277e-02  1.661e-02   1.972 0.051690 .  
baths:new   -1.043e+02  5.181e+01  -2.014 0.047075 *  
baths:size  -3.083e-03  1.590e-02  -0.194 0.846715    
new:size     1.053e-01  3.069e-02   3.431 0.000915 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian 

Analisando o p-value de cada variável vemos que existem umas que são mais relevantes e outras não. Note no modelo 2 (modN2) que as interações de segunda ordem mais relevantes são __beds:size, baths:new e new:size__. Isso nos demonstra que existe relações entre variáveis que devemos considerar em nossa modelagem. Tal fato ocorre pois uma variável acaba afetando a outra.

Vamos testar se há igualdade dos desvios para ambos os modelos por meio do teste F. Isso nos demonstrará se os modelos são estatísticamente iguais ou não.

In [3]:
#Testando se os desvios são iguais
anova(modN1,modN2,test='F')

Resid. Df,Resid. Dev,Df,Deviance,F,Pr(>F)
95,279624.1,NA,NA,NA,NA
89,217916.4,6,61707.71,4.200377,0.0009127911


Pelo resultado do p-value vemos que deve-se rejeitar a hipótese nula de que ambos possuem desvios iguais. Sabendo que os modelos são diferentes iremos criar um novo que contenha as interações mais relevantes. Após esse passo iremos remove-las individualmente para verificar o seu efeito no modelo.

In [4]:
#Modelo com as interações mais relevantes
modN3 = with(casas,lm(price ~ beds + baths + new + size + beds:size + baths*new + new*size))

#Descrição do modelo 3
summary(modN3)

#Removendo uma interação
summary(update(modN3,.~. - beds:size))

#Removendo uma interaçã
summary(update(modN3,.~. - baths:new))

#Removendo outra interação
summary(update(modN3,.~. - new:size))


Call:
lm(formula = price ~ beds + baths + new + size + beds:size + 
    baths * new + new * size)

Residuals:
     Min       1Q   Median       3Q      Max 
-144.342  -28.045   -3.326   18.060  180.538 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.356e+02  5.419e+01   2.503 0.014073 *  
beds        -5.505e+01  1.762e+01  -3.125 0.002383 ** 
baths        1.228e+01  1.218e+01   1.008 0.316002    
new          9.072e+01  7.754e+01   1.170 0.245019    
size        -3.178e-03  3.232e-02  -0.098 0.921879    
beds:size    3.086e-02  9.059e-03   3.406 0.000979 ***
baths:new   -1.115e+02  4.531e+01  -2.462 0.015684 *  
new:size     1.040e-01  2.865e-02   3.630 0.000466 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 48.7 on 92 degrees of freedom
Multiple R-squared:  0.7851,	Adjusted R-squared:  0.7688 
F-statistic: 48.02 on 7 and 92 DF,  p-value: < 2.2e-16



Call:
lm(formula = price ~ beds + baths + new + size + baths:new + 
    new:size)

Residuals:
     Min       1Q   Median       3Q      Max 
-177.127  -29.451   -3.575   16.559  184.867 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -28.24361   26.30667  -1.074 0.285767    
beds          -4.38277    9.96498  -0.440 0.661090    
baths         14.68847   12.83526   1.144 0.255401    
new           58.10608   81.21488   0.715 0.476117    
size           0.09863    0.01296   7.610 2.19e-11 ***
baths:new   -103.91647   47.76275  -2.176 0.032114 *  
new:size       0.10757    0.03022   3.560 0.000586 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 51.4 on 93 degrees of freedom
Multiple R-squared:  0.758,	Adjusted R-squared:  0.7424 
F-statistic: 48.55 on 6 and 93 DF,  p-value: < 2.2e-16



Call:
lm(formula = price ~ beds + baths + new + size + beds:size + 
    new:size)

Residuals:
     Min       1Q   Median       3Q      Max 
-149.708  -27.820   -1.907   16.914  173.827 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)   
(Intercept) 138.824314  55.629290   2.496  0.01434 * 
beds        -53.990032  18.087758  -2.985  0.00362 **
baths         4.811920  12.114243   0.397  0.69212   
new         -58.398959  49.710414  -1.175  0.24308   
size          0.005414   0.032989   0.164  0.86999   
beds:size     0.029752   0.009291   3.202  0.00187 **
new:size      0.055025   0.021174   2.599  0.01088 * 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 50 on 93 degrees of freedom
Multiple R-squared:  0.7709,	Adjusted R-squared:  0.7562 
F-statistic: 52.17 on 6 and 93 DF,  p-value: < 2.2e-16



Call:
lm(formula = price ~ beds + baths + new + size + beds:size + 
    baths:new)

Residuals:
     Min       1Q   Median       3Q      Max 
-177.362  -32.645   -1.494   19.940  157.986 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)   
(Intercept) 143.548384  57.581832   2.493  0.01444 * 
beds        -60.506512  18.669856  -3.241  0.00165 **
baths         2.376277  12.625038   0.188  0.85111   
new          56.149019  81.836558   0.686  0.49435   
size          0.010772   0.034121   0.316  0.75294   
beds:size     0.032064   0.009627   3.331  0.00124 **
baths:new     2.622243  34.682165   0.076  0.93989   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 51.78 on 93 degrees of freedom
Multiple R-squared:  0.7543,	Adjusted R-squared:  0.7385 
F-statistic: 47.59 on 6 and 93 DF,  p-value: < 2.2e-16


Analisando os modelos que possuiram alguma iteração retirada, note que o segundo possui um valor de adjusted R-squared (0.7562) menor comparado aos outros. 

Além disso, com relação ao modelo que possui todos os parâmetros veja que seu valor de adjusted R-squared é menor, mas como essa diferença é muito pequena e temos menos parâmetros no primeiro, iremos escolhe-lo.

In [5]:
modN4 = update(modN3,.~. - baths:new - baths)
summary(modN4)


Call:
lm(formula = price ~ beds + new + size + beds:size + new:size)

Residuals:
     Min       1Q   Median       3Q      Max 
-150.013  -27.452   -2.112   16.851  171.296 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)   
(Intercept) 143.470985  54.141190   2.650  0.00945 **
beds        -53.637336  17.984834  -2.982  0.00364 **
new         -56.685782  49.300599  -1.150  0.25314   
size          0.006840   0.032645   0.210  0.83448   
beds:size     0.030021   0.009225   3.254  0.00158 **
new:size      0.054409   0.021022   2.588  0.01118 * 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 49.78 on 94 degrees of freedom
Multiple R-squared:  0.7706,	Adjusted R-squared:  0.7584 
F-statistic: 63.14 on 5 and 94 DF,  p-value: < 2.2e-16


Vamos realizar uma interpretação de parâmetros nesse modelo para verificar como as interação afetam o valor da predição. Considere os seguintes cenários:

a) Aumento de 100 pés quadrados no valor predito de uma casa antiga de dois quartos. <br>
b) Aumento de 100 pés quadrados no valor predito de uma casa nova de 3 quartos.

Considerando que:

b0: intercept <br>
b1: beds <br>
b2: new <br>
b3: size  <br>
b4: beds:size <br>
b5: new:size

Nosso modelo é:

__y= b0 + x1*b1 + x2*b2 + x3*b3 + x1*x3*b4 + x2*x3*b5__


a) Resolução:

__y = b0 + 2*b1 + x3*b3 + 2*x3*b4 <br>
y*= b0 + 2*b1 + (x3+100)*b3 + 2*(x3+100)*b4 __<br>


__y*-y = 100 * b3 + 2*100*b4 = 100(b3 + 2*b4) = 100 * (0.006840 + 2*0.030021) = 6.6882__

Portanto, um aumento de 100 pés em uma casa antiga de dois quartos implica em um aumento de 6.6882 no seu preço.

b) Resolução:

__y= b0 + 3*b1 + b2 + x3*b3 + 3*x3*b4 + x3*b5 <br>
y* = b0 + 3*b1 + b2 + (x3+100)*b3 + 3*(x3+100)*b4 + (x3+100)*b5__

__y*-y = 100*b3 + 3*100*b4 + 100*b5 = 100 * (b3 + 3*b4 + b5) = 100 * (0.006840 + 3*0.030021+0.054409+ = 15.1312__

Portanto, um aumento de 100 pés em uma casa nova de três quartos implica em um aumento de 15.1312 no seu preço.

Pelos itens _a_ e _b_ note como os valores de b4 e b5 afetaram no aumento do preço. Sem eles tal aumento seria com toda certeza bem diferente.